# Test de création d'isochrones avec de nouvelles formes

On se base sur l'API de Navitia pour obtenir les points qui permettent de composer les isochrones

In [1]:
# Add module to path
import os
import sys

add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

In [2]:
from dotenv import load_dotenv
import docopt
import json
from pathlib import Path
from csv_to_json import csv_to_json
from isos_and_intersections import GetIso

param_csv = os.path.join(add_path, "params/test_journeys.csv")
param_json = os.path.join(add_path, "params/test_journeys.json")
places_cache = os.path.join(add_path, "params/places_cache.json")

#Parameters
env_path = Path(add_path) / '.env'
load_dotenv(dotenv_path=env_path)
TOKEN = os.getenv("NAVITIA_TOKEN")

columns_with_array_of_str = [
    "colors_iso",
    "addresses",
    "excluded_modes",
    "durations"
    ]

#JSON INPUT
json_file = csv_to_json(param_csv, param_json, ";", columns_with_array_of_str)

gdf_global = GetIso(param_json, places_cache, api="navitia", token=TOKEN).get_all_isos()

In [3]:
from geo_functions import gdf_to_geojson

columns_lines = list(gdf_global["journeys_details_lines"].columns)
columns_pts = list(gdf_global["journeys_details_pts"].columns)
geo_source_lines = gdf_to_geojson(gdf_global["journeys_details_lines"], columns_lines, "epsg:3857")
geo_source_pts = gdf_to_geojson(gdf_global["journeys_details_pts"], columns_pts, "epsg:3857")

In [9]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.tile_providers import STAMEN_TONER_BACKGROUND
from bokeh.models import GeoJSONDataSource, ColumnDataSource
from bokeh.palettes import Viridis
import json


output_notebook()

p = figure()
p.width = 800
p.add_tile(STAMEN_TONER_BACKGROUND, alpha=0.2)


source_lines = GeoJSONDataSource(geojson=geo_source_lines)
p.multi_line(
    xs='xs',
    ys='ys',
    line_width=2,
    line_alpha=0.5,
    source=source_lines
)

source_pts = GeoJSONDataSource(geojson=geo_source_pts)
p.circle(
    x='x',
    y='y',
    size = 10,
    source=source_pts
)
# geojsons_path = "./variation_journee"
# geojsons = get_files(geojsons_path)
# colors = Viridis[len(geojsons)]

# for i,geojson in enumerate(geojsons):
#     name_legend = geojson.split("\\")[-1].split(".")[0]
#     geojson = json.load(open(geojson))
#     geo_source = GeoJSONDataSource(geojson=json.dumps(geojson))
#     p.patches(
#         xs='xs', 
#         ys='ys', 
#         line_color= colors[i], 
#         fill_alpha=0.0, 
#         line_alpha=0.8, 
#         line_width=2, 
#         source=geo_source, 
#         legend=name_legend)

# p.legend.click_policy="hide"

show(p)

Loading BokehJS ...